In [2]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.impute import SimpleImputer
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

import pandas as pd
import numpy as np
import scipy as sci
from itertools import chain
import pdb
import os
import argparse
import sys

from models import *
from utils import *

%matplotlib inline
%load_ext autoreload
%autoreload 2


In [3]:
# absolute paths to data and output
# home_dir = "/scratch/users/gmachi/dl_multiomics/cs229_dl_multiomics"

In [16]:
def str2bool(v):
    if v.lower() in ('yes', 'true', 't', 'y', '1'):
        return True
    elif v.lower() in ('no', 'false', 'f', 'n', '0'):
        return False
    else:
        raise argparse.ArgumentTypeError('Boolean value expected.')


def run_ungrouped_models(X, y, target_names, e_size, args):

    # fit / train the models on single omic -> rna seq
    pca_loadings, pca_transformed = run_pca(X.values, e_size)
    fa_loadings, fa_transformed = run_fa(X.values, e_size)
    ael_loadings, ael_transformed = run_vanilla_autoencoder(X.values, "linear", e_size)
    aer_loadings, aer_transformed = run_vanilla_autoencoder(X.values, "relu", e_size)
    
    # tsne_loadings, tsne_transformed = run_tsne(X.values, e_size) #<--- e_size > 4 is an issue
    # umap_transformed = run_umap(rna.values, e_size)
    # aes_loadings, aes_transformed = run_basic_autoencoder(X.values, "sigmoid", e_size)

    # transpose AE models for consistency
    ael_loadings = ael_loadings.T
    aer_loadings = aer_loadings.T

    method_strings = ["pca", "fa", "ael", "aer"]
    loadings = [pca_loadings, fa_loadings, ael_loadings, aer_loadings]
    transformed = [pca_transformed, fa_transformed, ael_transformed, aer_transformed]

    # write to df and to csv
    print("\nloading dims:\n---------------------------")
    for i, l in enumerate(loadings):
        print(method_strings[i] + ": ", l.shape)
        df = pd.DataFrame.from_records(l)
        outfile = home_dir + "/model_outputs/" + args.dataset + "/loadings_" + args.num_factors + "/" + method_strings[i] + "_loadings.txt"
        df.to_csv(outfile, header=True, index=None, sep=',', mode='a')

    print("\ntransformed dims:\n---------------------------")
    for i, l in enumerate(transformed):
        print(method_strings[i] + ": ", l.shape)
        df = pd.DataFrame.from_records(l)
        outfile = home_dir + "/model_outputs/" + args.dataset + "/transformed_" + args.num_factors + "/" + method_strings[i] + "_transformed.txt"
        df.to_csv(outfile, header=True, index=None, sep=',', mode='a')

    return transformed, method_strings


def run_grouped_models(X, y, target_names, e_size, args):

    # added hyperparam for maui
    h = X.shape[1] # --> can also pick 1100 like Maui creators

    mofa_loadings, mofa_transformed = run_mofa(X, args, e_size)
    maui_loadings, maui_transformed = run_maui(X, args, e_size, h)
    # oivae_encodings = run_oivae(X, args, e_size)

    # get comparable shapes
    maui_loadings = maui_loadings.T
    mofa_transformed = mofa_transformed.T

    method_strings = ["mfa", "maui"]
    loadings = [mofa_loadings, maui_loadings]
    transformed = [mofa_transformed, maui_transformed]

    # write to df and to csv
    print("\nloading dims:\n---------------------------")
    for i, l in enumerate(loadings):
        print(method_strings[i] + ": ", l.shape)
        df = pd.DataFrame.from_records(l)
        outfile = home_dir + "/model_outputs/" + args.dataset + "/loadings_" + args.num_factors + "/" + method_strings[i] + "_loadings.txt"
        df.to_csv(outfile, header=True, index=None, sep=',', mode='a')

    print("\ntransformed dims:\n---------------------------")
    for i, l in enumerate(transformed):
        print(method_strings[i] + ": ", l.shape)
        df = pd.DataFrame.from_records(l)
        outfile = home_dir + "/model_outputs/" + args.dataset + "/transformed_" + args.num_factors + "/" + method_strings[i] + "_transformed.txt"
        df.to_csv(outfile, header=True, index=None, sep=',', mode='a')

    return transformed, method_strings


In [22]:
def runner(args):

    # read the data
    if args.dataset == "ipop":
        X, y, target_names = load_data(args.dataset)
        Xr, yr, target_names_r = X.loc[:, X.columns.str.startswith('rnaseq')], y, target_names
        
    elif args.dataset == "motrpac":
        Xr, X, yr, y, target_names_r, target_names = load_data(args.dataset)
        
    elif args.dataset == "arabidop1" or args.dataset == "arabidop2":
        X, y, target_names = load_data(args.dataset)
        # not supporting single omics here
        
    elif args.dataset == "tcga":
        X, y, target_names = load_data(args.dataset)
        # not supporting single omics here

    e_size = int(args.num_factors)  # encoding size / number of components (L)

    if args.mode == "run":
        if args.omic == 'single':

            try:
                encs_by_method, method_strings = run_ungrouped_models(Xr, yr, target_names_r, e_size, args.verbosity)
            except:
                print("no single-omics data detected")
                return

            # for i, encs in enumerate(encs_by_method):
                # prediction_task(encs, yr, method_strings[i], args)

        # fit / train the models on multiple (all 5) omics
        elif args.omic == 'multi':

            # Naive approach with stacked data! (ungrouped)
            #-----------------------------------
            encs_by_method_u, method_strings_u = run_ungrouped_models(X, y, target_names, e_size, args)

            # for i, encs in enumerate(encs_by_method_u):
            #     prediction_task(encs, y, method_strings_u[i], args)

            # Grouped analysis!
            #-------------------
            encs_by_method_g, method_strings_g = run_grouped_models(X, y, target_names, e_size, args)
            
            # for i, encs in enumerate(encs_by_method_g):
            #     prediction_task(encs, y, method_strings_g[i], args)

    elif args.mode == "analysis":
        print("run analysis script instead.")
        pass

    elif args.mode == "maui_only":

        args.num_factors = str(70)
        e_size = 70
        h = X.shape[1]

        loadings, transformed = run_maui(X, args, encoding_dim=e_size, hidden_dim=h)  # used to be default h = 1100
        print(loadings.shape)
        print(transformed.shape)

        # write to df and to csv
        print("\nloading dims:\n---------------------------")
        print("maui" + ": ", loadings.shape)
        df = pd.DataFrame.from_records(loadings)
        outfile = home_dir + "/model_outputs/" + args.dataset + "/loadings_" + args.num_factors + "/" + "maui" + "_loadings.txt"
        df.to_csv(outfile, header=True, index=None, sep=',', mode='a')

        print("\ntransformed dims:\n---------------------------")
        print("maui" + ": ", transformed.shape)
        df = pd.DataFrame.from_records(transformed)
        outfile = home_dir + "/model_outputs/" + args.dataset + "/transformed_" + args.num_factors + "/" + "maui" + "_transformed.txt"
        df.to_csv(outfile, header=True, index=None, sep=',', mode='a')


## Create Run object

In [23]:
class RunObj:
  def __init__(self, dataset, num_factors, omic, mode, cached_mfa, cached_vae, verbosity):
    self.dataset = dataset
    self.num_factors = num_factors
    self.omic = omic
    self.mode = mode
    self.cached_mfa = cached_mfa
    self.cached_vae = cached_vae
    self.verbosity = verbosity

# iPOP dataset

In [19]:
ipop = RunObj("ipop", "30", "multi", "run", False, False, False)
runner(ipop)

before drop na (589, 11494)
after drop na (589, 11494)
before cv filter (589, 11494)
after cv filter (589, 10835)
hi, cached mofa

    ###########################################################
    ###                 __  __  ___  _____ _                ### 
    ###                |  \/  |/ _ \|  ___/ \               ### 
    ###                | |\/| | | | | |_ / _ \              ### 
    ###                | |  | | |_| |  _/ ___ \             ### 
    ###                |_|  |_|\___/|_|/_/   \_\            ### 
    ###                                                     ###
    ########################################################### 
   
 
    
Loaded view 0 with 589 samples and 9883 features...
Loaded view 1 with 589 samples and 237 features...
Loaded view 2 with 589 samples and 643 features...
Loaded view 3 with 589 samples and 48 features...
Loaded view 4 with 589 samples and 24 features...

##############################################
## Doing sanity checks and parsing the

KeyboardInterrupt: 

# TCGA dataset

In [26]:
tcga = RunObj("tcga", "70", "multi", "run", True, True, False)
runner(tcga)


loading dims:
---------------------------
pca:  (70, 1300)
fa:  (70, 1300)
ael:  (70, 1300)
aer:  (70, 1300)

transformed dims:
---------------------------
pca:  (573, 70)
fa:  (573, 70)
ael:  (573, 70)
aer:  (573, 70)
hi, cached mofa

    ###########################################################
    ###                 __  __  ___  _____ _                ### 
    ###                |  \/  |/ _ \|  ___/ \               ### 
    ###                | |\/| | | | | |_ / _ \              ### 
    ###                | |  | | |_| |  _/ ___ \             ### 
    ###                |_|  |_|\___/|_|/_/   \_\            ### 
    ###                                                     ###
    ########################################################### 
   
 
    
Loaded view 0 with 573 samples and 1000 features...
Loaded view 1 with 573 samples and 200 features...
Loaded view 2 with 573 samples and 100 features...

##############################################
## Doing sanity checks and parsi

# Pioneer Dataset

In [ ]:
pioneer = RunObj("pioneer", 20, "multi", "run", False, False, False)
runner(pioneer)